# cambio de tipo de coordenada UTM


In [7]:
import pandas as pd
from pyproj import Proj, transform

Para poder trabajar con los datos en GEE lo primero qeu se debe de hacer es crear unas cordenadas que puedan leerse por GEE. Para ello pasaremos las coordenadas ``'UTM'`` a coordenadas de ``latitud/longuitud``

In [8]:
!pip install pyproj

### Introduccion de columna con `º, min, seg`

In [89]:

input_file = 'fechaXY.csv' 
output_file = 'coordenadas_geograficas_con_dms_grid.csv'
df = pd.read_csv(input_file, encoding='utf-8-sig', sep=';')

df.tail()

,ET_ID,X,Y,Date,Time
10521,10717,760226.4911,4455304.996,1995-04-17,5:00:00
10522,10718,737612.7840,4403833.345,1995-06-09,13:30:00
10523,10719,737425.1413,4438759.591,1995-05-30,17:24:00
10524,10720,733435.7536,4327179.033,1995-05-28,14:20:00
10525,10721,708406.2547,4323244.435,1995-04-30,14:20:00


In [90]:


# Definir la proyección UTM para la zona 30N (para Valencia, España)
utm_proj = Proj(proj='utm', zone=30, datum='WGS84', north=True)

# Definir la proyección geográfica (WGS84)
geo_proj = Proj(proj='latlong', datum='WGS84')

# Grados decimales
def dec_to_dms(degrees):
    is_negative = degrees < 0
    degrees = abs(degrees)
    
    # grados, minutos y segundos
    d = int(degrees)  # Grados
    m = int((degrees - d) * 60)  # Minutos
    s = (degrees - d - m / 60) * 3600  # Segundos
    s = round(s, 2)  # Redondear a dos decimales
    
    if is_negative:
        return f"-{d}° {m}' {s}\""
    else:
        return f"{d}° {m}' {s}\""



latitudes = []
longitudes = []
latitudes_dms = []
longitudes_dms = []

# Convertir las coordenadas 
for index, row in df.iterrows():
    x, y = row['X'], row['Y']
    lon, lat = transform(utm_proj, geo_proj, x, y)
    

    lat_dms = dec_to_dms(lat)
    lon_dms = dec_to_dms(lon)
    

    latitudes.append(lat)
    longitudes.append(lon)
    latitudes_dms.append(lat_dms)
    longitudes_dms.append(lon_dms)

# Agregar las coordenadas geográficas y en DMS 
df['Longitude'] = longitudes
df['Latitude'] = latitudes
#df['Longitude_DMS'] = longitudes_dms
#df['Latitude_DMS'] = latitudes_dms

# Guardar el DataFrame con las coordenadas geográficas y DMS 
df.to_csv(output_file, index=False)

print(f"Transformación completada. El archivo se ha guardado en {output_file}")


/var/folders/mq/qldjlz6d2qnfrl2d278n8dw40000gn/T/ipykernel_1182/3998465395.py:33: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, geo_proj, x, y)


Transformación completada. El archivo se ha guardado en coordenadas_geograficas_con_dms_grid.csv


### Modificar formato de DateTime segun ISO `2003-01-09T15:20:00.000Z`

In [91]:

file_path = "coordenadas_geograficas_con_dms_grid.csv"  
output_file_path = "coordenadas_geograficas_con_dms_G(!).csv" 

df = pd.read_csv(file_path)

if 'Date' not in df.columns or 'Time' not in df.columns:
    raise ValueError("El archivo CSV debe contener las columnas 'Date' y 'Time'.")


df['DateTime'] = pd.to_datetime(df['Date'] + 'T' + df['Time'], format='%Y-%m-%dT%H:%M:%S')


df['DateTime'] = df['DateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')  # Incluye milisegundos
df['DateTime'] = df['DateTime'].str[:-3] + 'Z'  # Ajusta a 3 decimales y agrega Z para UTC

# Guardar el archivo CSV con la columna DateTime
df.to_csv(output_file_path, index=False)

print(f"Archivo procesado guardado en: {output_file_path}")


Archivo procesado guardado en: coordenadas_geograficas_con_dms_G(!).csv


### Introduccion de indice

In [ ]:
import pandas as pd

# Cargar el archivo CSV
file_path = "coordenadas_geograficas_con_dms_G(!).csv"
df = pd.read_csv(file_path)

# Añadir un índice
df['Index'] = df.index

# Guardar el archivo modificado
output_path = "coordenadasG.csv"
df.to_csv(output_path, index=False)
print(f"Archivo con índice guardado en: {output_path}")


Archivo con índice guardado en: coordenadas.csv
